# Theoretical Modelling of Composite Galaxies 
This code is intended to output a dataframe containing the colours of the theoretical galaxy+AGN composites.



In [142]:
# Read in an AGN template
# Read in all required libraries
# Import in all of the required libraries
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
import pandas as pd
import os
from astLib import astSED
import astropy.io.fits as fits
import matplotlib.path as mpath
from carf import * # custom module for functions relating to the project

# So that we can change the helper functions without reloading the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
# In addition also use astSED to create filters

# Read in all filters
# UVJ Filters
pb_U_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.U.dat')
pb_V_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.V.dat')
pb_J_path = os.path.join('datasets', 'Filters', '2MASS_2MASS.J.dat')
# Spitzer filters
pb_f3_6_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I1.dat')
pb_f4_5_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I2.dat')
pb_f5_8_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I3.dat')
pb_f8_0_path = os.path.join('datasets', 'filters', 'Spitzer_IRAC.I4.dat')
# ugr filters
pb_u_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.u_SDSS.dat')
pb_g_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.g_SDSS.dat')
pb_r_path = os.path.join('datasets', 'filters', 'Paranal_OmegaCAM.r_SDSS.dat')


# Load all of the filters 
pb_U = astSED.Passband(pb_U_path, normalise=False)
pb_V = astSED.Passband(pb_V_path, normalise=False)
pb_J = astSED.Passband(pb_J_path, normalise=False)
pb_f3_6 = astSED.Passband(pb_f3_6_path, normalise=False)
pb_f4_5 = astSED.Passband(pb_f4_5_path, normalise=False)
pb_f5_8 = astSED.Passband(pb_f5_8_path, normalise=False)
pb_f8_0 = astSED.Passband(pb_f8_0_path, normalise=False)
pb_u = astSED.Passband(pb_u_path, normalise=False)
pb_g = astSED.Passband(pb_g_path, normalise=False)
pb_r = astSED.Passband(pb_r_path, normalise=False)

filter_set = {'U': pb_U, 'V':pb_V, 'J':pb_J, 'IRAC3.6': pb_f3_6,'IRAC4.5': pb_f4_5,'IRAC5.8': pb_f5_8, 'IRAC8.0':pb_f8_0, 'u': pb_u, 'g': pb_g, 'r':pb_r}



In [144]:
# Skirtor models
skirtor_folderpath = os.path.join('datasets', 'Templates', 'Skirtor')

# Swire templates
swire_folderpath = os.path.join('datasets', 'Templates', 'SWIRE')

# Brown templates
brown_folderpath = os.path.join('datasets', 'Templates', 'Brown', '2014','Rest')


In [145]:
# Read in the AGN templates
type1_agn, type1_params = create_type1_skirtor_agn(skirtor_folderpath)
type2_agn, type2_params = create_type2_skirtor_agn(skirtor_folderpath)

# Read in the template set of choice
swire_templates, template_names = read_swire_templates(swire_folderpath)

# Read in the brown templates
brown_templates, brown_template_names = read_brown_galaxy_templates(brown_folderpath)

In [146]:
# Choose running parameters

# Template set and AGN type
agn_model_name = 'Type1AGN'
template_set_name = 'Brown'

# Setup how many alpha values we want to explore
alpha_values = np.linspace(0, 1, 11)

In [147]:
# Choose an AGN model
if agn_model_name == 'Type1AGN':
    agn_model = type1_agn
elif agn_model_name == 'Type2AGN':
    agn_model = type2_agn
else:
    print('AGN model not recognised')

if template_set_name == 'SWIRE':
    template_set = swire_templates
    template_names = template_names
elif template_set_name == 'Brown':
    template_set = brown_templates
    template_names = brown_template_names
else:
    print('Template set not recognised')

In [148]:
# Check that all of the column names are easily able to be viewed
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100
0,Arp_118,0.0,-21.198908,-21.315776,-21.421279,-21.517432,-21.605757,-21.687435,-21.763396,-21.834388,...,-23.454779,-23.475153,-23.495151,-23.514788,-23.534076,-23.553028,-23.571654,-23.589966,-23.607974,-23.625689
1,Arp_118,0.1,-20.237172,-20.287567,-20.335726,-20.381839,-20.426074,-20.468576,-20.509478,-20.548894,...,-21.492481,-21.513877,-21.534859,-21.555444,-21.575645,-21.595478,-21.614955,-21.634088,-21.65289,-21.671372
2,Arp_118,0.2,-20.201401,-20.302567,-20.395105,-20.480372,-20.559427,-20.633115,-20.702117,-20.766996,...,-21.460665,-21.504041,-21.545751,-21.585917,-21.624651,-21.66205,-21.698203,-21.733192,-21.767088,-21.799958
3,Arp_118,0.3,-17.401194,-17.657027,-17.863926,-18.037638,-18.187352,-18.318902,-18.436221,-18.542091,...,-19.901468,-19.939198,-19.975661,-20.010938,-20.045106,-20.078231,-20.110376,-20.141596,-20.171943,-20.201465
4,Arp_118,0.4,-20.094718,-20.244207,-20.375583,-20.492764,-20.598521,-20.694885,-20.783389,-20.865219,...,-21.546386,-21.601016,-21.653028,-21.702663,-21.750127,-21.795603,-21.83925,-21.881211,-21.921609,-21.960559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,UM_461,0.3,24.632331,23.90264,23.470149,23.161691,22.921756,22.725366,22.559117,22.414976,...,22.184383,22.046727,21.924576,21.814789,21.71509,21.623782,21.53956,21.461404,21.388498,21.320181
1020,UM_461,0.4,23.150962,22.830091,22.582733,22.381405,22.211634,22.064854,21.935575,21.820064,...,21.246951,21.163347,21.085722,21.013279,20.945369,20.881457,20.8211,20.763921,20.709604,20.657876
1021,UM_461,0.5,23.503862,23.349919,23.215117,23.095218,22.987253,22.88906,22.799015,22.71587,...,21.826568,21.780132,21.735601,21.692825,21.65167,21.612019,21.573765,21.536813,21.501077,21.46648
1022,UM_461,0.6,26.405769,25.136712,24.567461,24.195932,23.919636,23.699581,23.516697,23.360224,...,23.121267,22.974438,22.845121,22.72958,22.62516,22.529908,22.442342,22.361315,22.285917,22.215416


In [149]:
# Create all of the composites
composites = generate_composite_set(agn_model, template_set, alpha_values)

In [225]:
# # # Save all the sampled colours to a data frame

# Create an empty data frame for each template that has the filters with the alpha values o.e u_0, u_10, u_20 etc for each filter
column_names = ['id', 'z'] # add an inital column for the redshift and for the ID
# filters 
for filter in filter_set.keys():
    # Check filter we are looking at
    print(filter)
    
    for alpha_val in alpha_values:
    # Add filter into a data frame
        column_names.append(filter + '_' + str(int(round(alpha_val, 2)*100)))

# Turn into a dataframe  
composite_fluxes = pd.DataFrame(columns=column_names)

# # redshif
# redshift_cutoff = 4

# # Try a more simple approach to this
# # I have a set of composites
# filter_sets = []

# redshifts = []
# ids = []

# # Create a list of empty lists for each filter
# for filter in filter_set.keys():
#     filter_sets.append([])
    
    
# # Add the redshifts and ids to the data frame
# for j in range(len(template_names)):
#     for redshift in np.arange(0, redshift_cutoff, 0.1):
#         redshifts.append(redshift)
#         ids.append(template_names[j])
    


U
V
J
IRAC3.6
IRAC4.5
IRAC5.8
IRAC8.0
u
g
r


In [226]:
composite_fluxes

,id,z,U_0,U_10,U_20,U_30,U_40,U_50,U_60,U_70,...,r_10,r_20,r_30,r_40,r_50,r_60,r_70,r_80,r_90,r_100


In [230]:
# To make this work for the redshifts we are going to need to create a new dataframe for each redshift
# This will be a list of dataframes

# for each redshift create a new data frame
redshifts = np.arange(0, 4, 0.1)

# Create a list of dataframes for each redshift
composite_fluxes_list = []


for redshift in redshifts:    
    # Create an empty data frame for each template that has the filters with the alpha values o.e u_0, u_10, u_20 etc for each filter
    flux_df = composite_fluxes.copy()
    
    
    for i in range(len(alpha_values)):
        # This will be the set of composites for the specific alpha value
        sed_alpha_data = composites[i]
        
        for j, sed_data in enumerate(sed_alpha_data):
            # Create an SED object using astSED
        
            #print(sed_data['lambda (Angstroms)'])
            wl = sed_data['lambda (Angstroms)']
            fl = sed_data['Total Flux (erg/s/cm^2/Angstrom)']
            
            # Create an SED object
            sed = astSED.SED(wavelength=wl, flux=fl, z=0.0)
            
            # Redshift the SED
            sed.redshift(redshift)
            
            # Add template name to the data frame in the id column
            flux_df.loc[j, 'id'] = template_names[j]
            
            # Only turn this on to output all seds
            # Only necessary to output the CSV for the rest frame
            
            if redshift == 0:
                sed_data.to_csv(f'outputs/composite_seds/{template_set_name}/'+template_names[j]+ f'{agn_model_name}'+ 'composite_' +str(int(round(alpha_values[i], 2)*100))+'.csv')
            
            # Calculate each filter value for the specific alpha value
            for filter in filter_set.keys():
                # Calculate the magnitude for the filter
                mag = astSED.SED.calcMag(sed, filter_set[filter], magType='AB')
                # Add the magnitude to the data frame
                flux_df.loc[j, filter + '_' + str(int(round(alpha_values[i], 2)*100))] = mag
                
    # Add the redshift to the data frame
    flux_df['z'] = redshift
    composite_fluxes_list.append(flux_df)

            
            
            

In [233]:
# We now want to join each of the data frames together
composite_fluxes = pd.concat(composite_fluxes_list)

In [241]:

# Output as required
composite_choice = f'{template_set_name}_theoretical_composite_fluxes_{agn_model_name}'
composite_fluxes.to_csv(f'outputs\composite_seds\{composite_choice}.csv')   

